In [1]:
%matplotlib inline

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!wget https://download.pytorch.org/tutorial/data.zip

!unzip data.zip

--2021-07-24 13:17:37--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.32.199.96, 13.32.199.123, 13.32.199.15, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.32.199.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip.3’

data.zip.3          100%[===================>]   2.75M  --.-KB/s    in 0.04s   

2021-07-24 13:17:37 (73.5 MB/s) - ‘data.zip.3’ saved [2882130/2882130]

Archive:  data.zip
replace data/eng-fra.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# Data Preprocessing

In [2]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['je l emporte .', 'i m winning .']


In [3]:
type(pairs)

list

In [4]:
pairs[0:5]

[['j ai ans .', 'i m .'],
 ['je vais bien .', 'i m ok .'],
 ['ca va .', 'i m ok .'],
 ['je suis gras .', 'i m fat .'],
 ['je suis gros .', 'i m fat .']]

In [5]:
help(input_lang)

Help on Lang in module __main__ object:

class Lang(builtins.object)
 |  Lang(name)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, name)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  addSentence(self, sentence)
 |  
 |  addWord(self, word)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [7]:
input_lang.n_words, output_lang.n_words

(4345, 2803)

In [8]:
input_size = input_lang.n_words

In [9]:
sample = random.choice(pairs)
sample

['tu n es pas different .', 'you re no different .']

In [11]:
src = sample[0]
trg = sample[1]

In [12]:
src = [input_lang.word2index[i] for i in src.split(" ")]
trg = [output_lang.word2index[i] for i in trg.split(" ")]

In [13]:
src.append(EOS_token), trg.append(EOS_token)

(None, None)

In [15]:
src = torch.tensor(src)
trg = torch.tensor(trg)

In [16]:
src, trg

(tensor([ 210,  245,  211,  246, 1442,    5,    1]),
 tensor([129,  78, 183, 373,   4,   1]))

In [17]:
src.shape[0]

7

# Initialising Dimensions

In [18]:
input_dim = input_lang.n_words
embedding_dim = 256
hidden_dim = 256
output_dim = output_lang.n_words

# Encoder (First two words)

In [24]:
encoder_embedding = nn.Embedding(input_dim, embedding_dim)
encoder_rnn = nn.LSTM(embedding_dim, hidden_dim)
encoder_outputs = torch.zeros((src.shape[0], 1, embedding_dim))
encoder_hidden = torch.zeros(1, 1, 256)
encoder_cell = torch.zeros(1, 1, 256)
for i in range(2):
  encoder_embedded = encoder_embedding(src[i]).reshape(1, 1, -1)
  encoder_output, (encoder_hidden, encoder_cell) = encoder_rnn(encoder_embedded, (encoder_hidden, encoder_cell))
  encoder_outputs[i] = encoder_output[0]


# Decoder (First two words)

In [29]:
trg = torch.cat((torch.tensor([SOS_token]), trg))
decoder_hidden = encoder_hidden
decoder_cell = encoder_cell
attention_weight_layer = nn.Linear(hidden_dim + embedding_dim, src.shape[0])
decoder_pre_rnn = nn.Linear(hidden_dim + embedding_dim, embedding_dim)
output_layer = nn.Linear(hidden_dim, output_lang.n_words)
for i in range(2):
  decoder_embedding = nn.Embedding(output_dim, embedding_dim)
  decoder_rnn = nn.LSTM(embedding_dim, hidden_dim)
  decoder_embedded = decoder_embedding(trg[i])  
  attention_weights = F.softmax(attention_weight_layer(torch.cat((decoder_hidden, decoder_embedded.reshape(1, 1, -1)), dim=2)), dim = 2)
  attn_applied = torch.bmm(attention_weights, encoder_outputs.squeeze(1).unsqueeze(0))  
  decoded_output, (decoded_hidden, decoded_cell) = decoder_rnn(decoder_pre_rnn(torch.cat((attn_applied, decoder_embedded.reshape(1, 1, -1)), dim = 2)), (decoder_hidden, decoder_cell))
  output = F.softmax(output_layer(decoded_output), dim = 2)
  print(f"Output predicted by word {i + 1} from decoder is '{output_lang.index2word[output.argmax(dim=2).item()]}'")

Output predicted by word 1 from decoder is 'lacking'
Output predicted by word 2 from decoder is 'lonely'
